In [1]:
import sys
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import pandas_gbq

import pandas as pd
import numpy as np

In [5]:
df = spark.read.parquet("/home/Archie/final-project/datasets/application_record.parquet", header=True, inferSchema=True)
df.printSchema()

root
 |-- ID: long (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: long (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- DAYS_BIRTH: long (nullable = true)
 |-- DAYS_EMPLOYED: long (nullable = true)
 |-- FLAG_MOBIL: long (nullable = true)
 |-- FLAG_WORK_PHONE: long (nullable = true)
 |-- FLAG_PHONE: long (nullable = true)
 |-- FLAG_EMAIL: long (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- CNT_FAM_MEMBERS: double (nullable = true)



In [6]:
rows = df.count()
cols = len(df.columns)

print(f'Dimensions of Data: {(rows,cols)}')
print(f'Rows of Data: {rows}')
print(f'Columns of Data: {cols}')

Dimensions of Data: (438557, 18)
Rows of Data: 438557
Columns of Data: 18


In [7]:
df.show(5)

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|    -12005|        -4542|         1

In [8]:
df_transform1 = df.withColumn("YEARS_BIRTH", floor(abs(df["DAYS_BIRTH"] / 365.25))) \
                    .withColumn("YEARS_EMPLOYED", floor(abs(df["DAYS_EMPLOYED"] / 365.25))) \
                    .drop("DAYS_BIRTH") \
                    .drop("DAYS_EMPLOYED")

In [9]:
df_transform1.show(5)

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+---------------+----------+----------+---------------+---------------+-----------+--------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|YEARS_BIRTH|YEARS_EMPLOYED|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+---------------+----------+----------+---------------+---------------+-----------+--------------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|         1|              1|  

In [10]:
df_transform2 = df_transform1.withColumn("CODE_GENDER", when(df.CODE_GENDER == "F", 1).otherwise(0)) \
                                .withColumn("FLAG_OWN_CAR", when(df["FLAG_OWN_CAR"] == "Y", 1).otherwise(0)) \
                                .withColumn("FLAG_OWN_REALTY", when(df["FLAG_OWN_REALTY"] == "Y", 1).otherwise(0))

In [11]:
df_transform2.show(5)

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+---------------+----------+----------+---------------+---------------+-----------+--------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|YEARS_BIRTH|YEARS_EMPLOYED|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+---------------+----------+----------+---------------+---------------+-----------+--------------+
|5008804|          0|           1|              1|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment|         1|              1|  

In [12]:
df_transform3 = df_transform2.na.drop("all")

In [13]:
rows = df_transform3.count()
cols = len(df_transform3.columns)

print(f'Dimensions of Data: {(rows,cols)}')
print(f'Rows of Data: {rows}')
print(f'Columns of Data: {cols}')

Dimensions of Data: (438557, 18)
Rows of Data: 438557
Columns of Data: 18


In [14]:
df_transform3_gbq = df_transform3.toPandas()

In [15]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/Archie/.google/credentials/google_credentials.json"

In [17]:
# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "data-fellowship-9-project"

# TODO: Set dataset_id to the full destination dataset ID.
table_id = 'final_project.raw_credit_card'

pandas_gbq.to_gbq(df_transform3_gbq, table_id, project_id=project_id)

100%|██████████| 1/1 [00:00<00:00, 18808.54it/s]
